## 11.1. Queries, Keys, and Values

到目前为止，我们所验证的所有网络都非常依赖于规模明确的输入。例如，ImageNet中的图像大小为$224 \times 224$ 像素，而且CNNs可根据大小进行网络调整. 甚至在自然语言处理过程中，RNNs的输入大小也需要被提前定义和固定. 实现可变的大小可以通过每个时间段上的顺序处理或者特别设计的卷积核进行实现（[Kalchbrenner等人，2014](https://d2l.ai/chapter_references/zreferences.html#id142)）当输入的大小随着信息内容的变化而变化时，这种方法可能会导致严重的问题，例如第10.7节中的文本转换（[Sutskever等人，2014](https://d2l.ai/chapter_references/zreferences.html#id275)）. 特别是对于长序列，很难跟踪网络已经生成或甚至查看的所有内容。即使像Yang等人（[2016](https://d2l.ai/chapter_references/zreferences.html#id274)）这样的显式跟踪启发式方法也只能提供有限的增益。

将其与数据集进行比较.在最简单的形式下，它们是键（$k$）和值（$v$）的集合。例如，我们的数据集 $\mathcal{D}$可能由元组组成，最后一个名字是键，第一个名字是值。\{("Zhang", "Aston"), ("Lipton", "Zachary"), ("Li", "Mu"), ("Smola", "Alex"), ("Hu", "Rachel"), ("Werness", "Brent")\}。例如，我们可以在数据集$\mathcal{D}$中对“Li”进行精确查询（$q$），该查询将返回值“Mu”。如果（“Li”、“Mu”）不是$\mathcal{D}$中的记录，则不会有有效答案。如果我们也允许近似匹配，我们将检索（“Lipton”，“Zachary”）。尽管如此，这个非常简单且微不足道的例子教会了我们许多有用的东西：
- 我们可以设计无论数据库大小如何都有效的查询$q$方式来实现键-值对的($k$,$v$)操作。
- 根据数据库的内容，相同的查询可以接收不同的答案。
- 在大型状态空间（数据库）上运行的“代码”可能非常简单(例如，精确匹配、近似匹配、前k项top-$k$).
- 不需要压缩或简化数据库就可以使操作有效。

显然，如果不是为了解释深度学习，我们不会在这里引入一个简单的数据库。事实上，这导致了过去十年在深度学习中引入的最令人兴奋的概念之一：*注意力机制*（[Bahdanau等人，2014](https://d2l.ai/chapter_references/zreferences.html#id10)）。稍后我们将介绍它在机器翻译中的具体应用。现在，只需考虑以下内容：表示为$m$个*键*和*值*的元组数据库$\mathcal{D} \stackrel{\mathrm{def}}{=} \{(\mathbf{k}_1, \mathbf{v}_1), \ldots (\mathbf{k}_m, \mathbf{v}_m)\}$。此外，用查询$\mathbf{q}$表示。然后我们可以将注意力定义$\mathcal{D}$为

$$\mathrm{Attention}(\mathbf{q}, \mathcal{D}) \stackrel{\mathrm{def}}{=} \sum_{i=1}^m \alpha(\mathbf{q}, \mathbf{k}_i) \mathbf{v}_i,$$

式中，$\alpha(\mathbf{q}, \mathbf{k}_i) \in \mathbb{R}$ ($i = 1, \ldots, m$)是标准化的注意力权重。这个操作本身通常被称为注意力汇聚/池化。“注意力”这个命名源于该操作中特别关注权重$\alpha$的重要性（即，大）。因此，注意力$\mathcal{D}$会生成数据库中包含的值的线性组合。事实上，这包含了上述示例作为一种特殊情况，其中除一个权重外，其他所有权重均为零。我们有一些特殊情况：

- 权重$\alpha(\mathbf{q}, \mathbf{k}_i)$为非负。在这种情况下，注意力机制的输出包含在由值跨越的凸面体中$\mathbf{v}_i$.
- 权重 $\alpha(\mathbf{q}, \mathbf{k}_i)$形成凸面体，即对所有$i$，满足$\sum_i \alpha(\mathbf{q}, \mathbf{k}_i) = 1$和$\alpha(\mathbf{q}, \mathbf{k}_i) \geq 0$
。这是深度学习中最常见的设置。
- 当其他权重全为0时，唯一的权重$\alpha(\mathbf{q}, \mathbf{k}_i)$为$1$，类似于传统数据库中的查询。
- 所有的权重是等价的，如对于所有的$i$，有$\alpha(\mathbf{q}, \mathbf{k}_i) = \frac{1}{m}$。这相当于整个数据库的平均值，也称为深度学习中的平均池化。

确保权重相加为1的常用策略是通过正则化方式

$$\alpha(\mathbf{q}, \mathbf{k}_i) = \frac{\alpha(\mathbf{q}, \mathbf{k}_i)}{{\sum_j} \alpha(\mathbf{q}, \mathbf{k}_j)}.$$

此外，可以借助于指数函数来确保权重是非负的。这意味着我们现在可以选择任何函数$a(\mathbf{q}, \mathbf{k})$，然后通过以下方式对其应用用于多指标模型的softmax操作

$$\alpha(\mathbf{q}, \mathbf{k}_i) = \frac{\exp(a(\mathbf{q}, \mathbf{k}_i))}{\sum_j \exp(a(\mathbf{q}, \mathbf{k}_j))}. $$

这种操作在所有的深度学习框架中都是现成的。它是可微分的，而且它的梯度永远不会消失，这些都是模型中理想的特性。不过请注意，上面介绍的注意力机制并不是唯一的选择。例如，我们可以设计一个非可分的注意力模型，可以使用强化学习方法进行训练（[Mnih等人，2014](https://d2l.ai/chapter_references/zreferences.html#id196)）。正如人们所期望的那样，训练这样一个模型是相当复杂的。因此，大部分的现代注意力研究都遵循图11.1.1中的框架。因此，我们将阐述的重点放在这个可分化机制的家族上。

![The attention mechanism computes a linear combination over values $\mathbf{v}_i$ via attention pooling,
where weights are derived according to the compatibility between a query $\mathbf{q}$ and keys $\mathbf{k}_i$.](../img/qkv.svg)

值得注意的是，在键和值的集合上执行的实际 "代码"，即查询，可以是相当简洁的，尽管操作的空间很大。这对网络层来说是一个理想的属性，因为它不需要太多的参数来学习。同样方便的是，注意力可以在任意大的数据库上操作，而不需要改变注意力集合操作的方式。

In [2]:
import torch
from d2l import torch as d2l

### 11.1.1 可视化

注意力机制的好处之一是它可以非常直观，特别是当权重为非负值且总和为1时。 在这种情况下，我们可以把大权重解释为模型选择相关成分的一种方式。虽然这是一个很好的直觉，但重要的是要记住，这只是一个直觉。不管怎么说，我们可能想在应用各种不同的查询时，将其对给定的键集的影响可视化。这个功能以后会很有用。

因此我们定义了show_heatmaps函数。请注意，它并不接受一个矩阵（注意力的权重）作为其输入，而是一个有4个轴的张量，允许不同的查询和权重的阵列。因此，输入矩阵的形状是（显示的行数、显示的列数、查询的数量、键的数量）。以后当我们想把用于设计Transformers的第11.5节的工作原理可视化时，这将会很方便。

In [3]:
#@save
def show_heatmaps(matrices, xlabel, ylabel, titles=None, figsize=(2.5, 2.5),
                  cmap='Reds'):
    """Show heatmaps of matrices."""
    d2l.use_svg_display()
    num_rows, num_cols, _, _ = matrices.shape
    fig, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize,
                                 sharex=True, sharey=True, squeeze=False)
    for i, (row_axes, row_matrices) in enumerate(zip(axes, matrices)):
        for j, (ax, matrix) in enumerate(zip(row_axes, row_matrices)):
            pcm = ax.imshow(matrix.detach().numpy(), cmap=cmap)
            if i == num_rows - 1:
                ax.set_xlabel(xlabel)
            if j == 0:
                ax.set_ylabel(ylabel)
            if titles:
                ax.set_title(titles[j])
    fig.colorbar(pcm, ax=axes, shrink=0.6);

作为一个快速的健全性检查，让我们可视化身份矩阵，表示只有当查询和键相同时，注意力权重才为1的情况。

In [4]:
attention_weights = torch.eye(10).reshape((1, 1, 10, 10))
show_heatmaps(attention_weights, xlabel='Keys', ylabel='Queries')

: 

: 

### 11.1.2 总结
注意力机制允许我们从许多键-值对中汇总数据。到目前为止，我们的讨论非常抽象，只是描述了一种汇集数据的方法。我们还没有解释这些神秘的查询、键和值可能来自何处。在这里，一些直觉可能会有所帮助：例如，在回归设置中，查询可能对应于应该执行回归的位置。键是观察过去数据的位置，值是（回归）值本身。这就是所谓的Nadaraya Watson估计量（[Nadaraya，1964，Watson，1964](https://d2l.ai/chapter_references/zreferences.html#id313)），我们将在下一节中进行研究。
通过设计，注意力机制提供了一种可微分的控制方法，通过该方法，神经网络可以从集合中选择元素，并在表示上构造相关的加权和。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
n_train = 50  # 训练样本数
x_train, _ = torch.sort(torch.rand(n_train) * 5)   # 排序后的训练样本

def f(x):
    return 2 * torch.sin(x) + x**0.8

y_train = f(x_train) + torch.normal(0.0, 0.5, (n_train,))  # 训练样本的输出
x_test = torch.arange(0, 5, 0.1)  # 测试样本
y_truth = f(x_test)  # 测试样本的真实输出
n_test = len(x_test)  # 测试样本数
n_test

50

In [3]:
def plot_kernel_reg(y_hat):
    d2l.plot(x_test, [y_truth, y_hat], 'x', 'y', legend=['Truth', 'Pred'],
             xlim=[0, 5], ylim=[-1, 5])
    d2l.plt.plot(x_train, y_train, 'o', alpha=0.5);

In [4]:
y_hat = torch.repeat_interleave(y_train.mean(), n_test)
plot_kernel_reg(y_hat)

: 

: 